In [ ]:
import pandas as pd; print("Ready!")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


plt.style.use('seaborn-v0_8')
%matplotlib inline

In [ ]:

cols = ['age','sex','cp','trestbps','chol','fbs','restecg',
        'thalach','exang','oldpeak','slope','ca','thal','target']

df = pd.read_csv('../data/heart.csv', header=None, names=cols)

print("Shape:", df.shape)
df.head()

#this commandes are for getting the information of the data and assigning the names to the columns 

In [ ]:
df.info()
df.describe() 

#this commandes are for visualizing the data and getting a better understanding of the data

In [ ]:
df = df.replace('?', np.nan)
df = df.apply(pd.to_numeric)

print("Missing values:")
print(df.isnull().sum())

#this commandes are for filling the missing values with the median and the mode

Missing values:
age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          4
thal        2
target      0
dtype: int64


In [9]:
df
df['ca'].fillna(df['ca'].median(), inplace=True)
df['thal'].fillna(df['thal'].mode()[0], inplace=True)

print("Missing values after imputation:", df.isnull().sum().sum())

Missing values after imputation: 0


In [ ]:
# Target distribution
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
sns.countplot(data=df, x='target', palette='Set2')
plt.title('Heart Disease Distribution')
plt.xticks([0,1], ['No Disease', 'Disease'])

# visualizing the data between the age and the disease
plt.subplot(1,2,2)
sns.histplot(data=df, x='age', hue='target', multiple='stack', bins=20, palette='Set1')
plt.title('Age Distribution by Disease')

plt.tight_layout()
plt.show()

# Correlation heatmap
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Feature Correlation Matrix')
plt.show()